In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
from pathlib import Path

folder_path = Path(r".\Revisions\SevereHypoDataset\Data Tables")

BDataCGM = pd.read_csv(folder_path / "BDataCGM.txt", sep='|')
BMedicalConditions = pd.read_csv(folder_path / "BMedicalConditions.txt", sep='|')
BSampleResults = pd.read_csv(folder_path / "BSampleResults.txt", sep='|')
BDemoLifeDiabHxMgmt = pd.read_csv(folder_path / "BDemoLifeDiabHxMgmt.txt", sep='|')

print(BDataCGM.head())
print(BMedicalConditions.head())

   RecID  PtID BCGMDeviceType BFileType  DeviceDaysFromEnroll  DeviceTm  \
0   7393   199         Dexcom   Visit 1                     0  17:42:44   
1   7394   199         Dexcom   Visit 1                     0  17:44:05   
2   7395   199         Dexcom   Visit 1                     1  10:08:13   
3   7396   199         Dexcom   Visit 1                     1  22:54:45   
4   7397   199         Dexcom   Visit 1                     2  09:58:59   

   Glucose  CalBG  
0      NaN    NaN  
1      NaN    NaN  
2      NaN    NaN  
3      NaN    NaN  
4      NaN    NaN  
   RecID  PtID                       MCLLTReal  MedCondTrt
0      1    36          Chronic kidney disease         NaN
1      2    36                    Hypertension         NaN
2      3    36                  Hypothyroidism  Medication
3      4    36  Diabetic peripheral neuropathy         NaN
4      5    36                      Depression  Medication


In [6]:
roster = pd.read_csv(folder_path / "BPtRoster.txt", sep='|')
num_rows = roster.shape[0]
print(num_rows)

203


In [7]:
comorbidity_counts = BMedicalConditions['MCLLTReal'].value_counts()

print(comorbidity_counts.head(10))

MCLLTReal
Hypertension                          125
Hyperlipidemia                         72
Hypothyroidism                         55
Depression                             38
Coronary artery disease                37
Diabetic peripheral neuropathy         31
Dyslipidemia                           29
Chronic kidney disease                 26
Proliferative diabetic retinopathy     26
Osteoporosis                           26
Name: count, dtype: int64


In [8]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
from joblib import Parallel, delayed

LOW = 70
HIGH = 180
SPIKE_THRESHOLD = 50
CONGA_LAG = 4

# --- Feature computation function ---
def compute_cgm_features_clean(group, conga_lag=CONGA_LAG, spike_threshold=SPIKE_THRESHOLD):
    values = pd.to_numeric(group['Glucose'], errors='coerce').values
    timestamps = pd.to_datetime(group['Timestamp'], errors='coerce')
    
    # Drop missing glucose or timestamps
    mask_valid = (~np.isnan(values)) & (~pd.isnull(timestamps))
    values = values[mask_valid]
    timestamps = (timestamps[mask_valid] - timestamps[mask_valid].min()).dt.total_seconds().values
    
    n = len(values)
    if n < 2:
        return pd.Series({'PtID': group['PtID'].iloc[0]})
    
    # --- Trend features ---
    tir_array = ((values >= LOW) & (values <= HIGH)).astype(int)
    slope_tir = linregress(timestamps, tir_array).slope * 86400
    slope_mean = linregress(timestamps, values).slope * 86400

    # --- Percentiles ---
    percentiles = np.percentile(values, [10, 25, 50, 75, 90])
    dist_features = {f'glucose_p{p}': val for p, val in zip([10, 25, 50, 75, 90], percentiles)}

    # --- Rolling stats ---
    rolling_window = max(2, n // 10)
    roll_series = pd.Series(values)
    rolling_std = roll_series.rolling(rolling_window).std().to_numpy()
    rolling_mean = roll_series.rolling(rolling_window).mean().to_numpy()
    rolling_cv = rolling_std / np.where(rolling_mean != 0, rolling_mean, np.nan)
    rolling_features = {
        'rolling_mean_std': np.nanstd(rolling_std),
        'rolling_cv_std': np.nanstd(rolling_cv)
    }

    # --- Glycemic variability ---
    diffs = np.diff(values)
    mage = np.mean(np.abs(diffs)[np.abs(diffs) > np.std(values)]) if len(diffs) > 0 else 0
    modd = np.mean(np.abs(diffs)) if len(diffs) > 0 else 0
    conga = np.std(values[conga_lag:] - values[:-conga_lag]) if n > conga_lag else 0
    gly_var_features = {'MAGE': mage, 'MODD': modd, f'CONGA_{conga_lag}': conga}

    # --- Hypo/Hyper events ---
    hypo = (values < LOW).astype(int)
    hyper = (values > HIGH).astype(int)
    hypo_count = np.sum(np.diff(np.concatenate([[0], hypo, [0]])) == 1)
    hyper_count = np.sum(np.diff(np.concatenate([[0], hyper, [0]])) == 1)
    hypo_durations = np.diff(np.where(np.concatenate([[0], hypo, [0]]) == 1)[0])[::2]
    hyper_durations = np.diff(np.where(np.concatenate([[0], hyper, [0]]) == 1)[0])[::2]
    event_features = {
        'hypo_count': hypo_count,
        'hyper_count': hyper_count,
        'avg_hypo_duration': np.mean(hypo_durations) if len(hypo_durations) > 0 else 0,
        'avg_hyper_duration': np.mean(hyper_durations) if len(hyper_durations) > 0 else 0
    }

    # --- Time-of-day ---
    hours = group['Timestamp'].dt.hour.values[mask_valid]
    day_mask = (hours >= 6) & (hours < 22)
    night_mask = ~day_mask
    day_values = values[day_mask]
    night_values = values[night_mask]
    time_features = {
        'TIR_day': np.mean((day_values >= LOW) & (day_values <= HIGH)) * 100 if len(day_values) > 0 else np.nan,
        'TIR_night': np.mean((night_values >= LOW) & (night_values <= HIGH)) * 100 if len(night_values) > 0 else np.nan,
        'TBR_day': np.mean(day_values < LOW) * 100 if len(day_values) > 0 else np.nan,
        'TBR_night': np.mean(night_values < LOW) * 100 if len(night_values) > 0 else np.nan,
        'CV_day': np.std(day_values) / np.mean(day_values) if len(day_values) > 0 and np.mean(day_values) != 0 else 0,
        'CV_night': np.std(night_values) / np.mean(night_values) if len(night_values) > 0 and np.mean(night_values) != 0 else 0
    }

    # --- Spike/fall ---
    spike_count = np.sum(np.abs(diffs) > spike_threshold)
    spike_features = {'rapid_spike_count': spike_count}

    # --- Interaction ---
    interaction_features = {'slope_CV_interaction': slope_mean * np.nanmean(values)}

    # --- Combine all features ---
    all_features = {
        'PtID': group['PtID'].iloc[0],
        'slope_TIR_per_day': slope_tir,
        'slope_mean_glucose_per_day': slope_mean,
        **dist_features,
        **rolling_features,
        **gly_var_features,
        **event_features,
        **time_features,
        **spike_features,
        **interaction_features
    }

    return pd.Series(all_features)


# =====================
# Parallelized application
# =====================
def compute_all_cgm_features(df_glucose, n_jobs=-1):
    # Build Timestamp safely from DeviceDaysFromEnroll + DeviceTm
    df_glucose['Timestamp'] = pd.to_datetime('2023-01-01') + \
                              pd.to_timedelta(df_glucose['DeviceDaysFromEnroll'], unit='D') + \
                              pd.to_timedelta(df_glucose['DeviceTm'], errors='coerce')
    
    df_glucose = df_glucose.sort_values(['PtID','Timestamp'])
    patients = [group for _, group in df_glucose.groupby('PtID')]
    results = Parallel(n_jobs=n_jobs)(
        delayed(compute_cgm_features_clean)(p) for p in patients
    )
    return pd.DataFrame(results)

# =====================
# Usage
# =====================
df_metrics = compute_all_cgm_features(BDataCGM)

In [11]:
# Temporarily display all columns and widen display
with pd.option_context('display.max_columns', None, 'display.width', 200):
    print(df_metrics.head())
    print(f"Number of columns: {df_metrics.shape[1]}")
    


   PtID  slope_TIR_per_day  slope_mean_glucose_per_day  glucose_p10  glucose_p25  glucose_p50  glucose_p75  glucose_p90  rolling_mean_std  rolling_cv_std       MAGE      MODD    CONGA_4  hypo_count  \
0   1.0          -0.021325                    6.072068         75.6        127.5        214.0        293.0        367.4         15.616631        0.126335        NaN  6.386350  28.986314        18.0   
1   2.0          -0.019459                    0.969768         69.0        107.0        152.0        204.0        252.0         12.953812        0.063429  99.500000  3.773408  18.230551        15.0   
2   3.0          -0.010997                    3.050239        120.0        156.0        191.0        229.0        283.6         12.908488        0.066222        NaN  4.958574  21.468171         7.0   
3   4.0           0.007940                    0.426055         68.3         86.0        100.0        120.0        149.0          8.876556        0.083210  39.250000  2.358564  11.322750        12.

In [12]:
# Step 1: Define the mapping of income categories to numeric midpoints
income_mapping = {
    'Less than $25,000': 12500,
    '$25,000 - $35,000': 30000,
    '$35,000 - less than $50,000': 42500,
    '$50,000 - less than $75,000': 62500,
    '$75,000 - less than $100,000': 87500,
    '$100,000 - less than $200,000': 150000,
    '$200,000 or more': 200000,
}

# Step 2: Create numeric AnnualInc in your demographics dataframe
BDemoLifeDiabHxMgmt['AnnualInc_num'] = BDemoLifeDiabHxMgmt['AnnualInc'].map(income_mapping)

# Step 3: Select relevant demographic columns to merge
demographics = BDemoLifeDiabHxMgmt[['PtID', 'Gender', 'Ethnicity', 'Race', 
                                    'EduLevel', 'EduLevelNoAns', 'EduLevelUnk', 'AnnualInc_num']].drop_duplicates(subset='PtID')

# Step 4: Merge into df_metrics
df_metrics = df_metrics.merge(demographics, on='PtID', how='left')

# Step 5: Optionally, rename the numeric column to replace the old one
df_metrics = df_metrics.rename(columns={'AnnualInc_num': 'AnnualInc'})

In [14]:
df_metrics.head()
print(f"Number of columns: {df_metrics.shape[1]}")

Number of columns: 32


In [15]:
lab_results = BSampleResults[['RecID', 'PtID', 'Analyte', 'ResultName', 'Value', 'Units']].copy()

lab_results_wide = lab_results.pivot_table(
    index='PtID',           # one row per patient
    columns='ResultName',   # each ResultName becomes a column
    values='Value',         # fill with the numeric result
    aggfunc='first'         # if multiple entries, take the first (or you could use mean)
).reset_index()

df_metrics = df_metrics.merge(lab_results_wide, on='PtID', how='left')

In [17]:
df_metrics.head()
print(f"Number of columns: {df_metrics.shape[1]}")

Number of columns: 36


In [11]:
hypertension_flag = (
    BMedicalConditions.groupby('PtID')['MCLLTReal']
    .apply(lambda x: int('Hypertension' in x.values))
    .reset_index()
    .rename(columns={'MCLLTReal': 'Hypertension'})
)

df_metrics = df_metrics.merge(hypertension_flag, on='PtID', how='left')

df_metrics['Hypertension'] = df_metrics['Hypertension'].fillna(0).astype(int)

In [12]:
df_metrics.head()

,PtID,slope_TIR_per_day,slope_mean_glucose_per_day,glucose_p10,glucose_p25,glucose_p50,glucose_p75,glucose_p90,rolling_mean_std,rolling_cv_std,...,Race,EduLevel,EduLevelNoAns,EduLevelUnk,AnnualInc,C-Peptide,Glucose,HbA1c,Serum Creatinine,Hypertension
0,1.0,-0.021325,6.072068,75.6,127.5,214.0,293.0,367.4,15.616631,0.126335,...,White,Associate Degree,NaN,NaN,87500.0,0.020,69,9.2,0.77,1
1,2.0,-0.019459,0.969768,69.0,107.0,152.0,204.0,252.0,12.953812,0.063429,...,White,Some college but no degree,NaN,NaN,87500.0,<0.017,295,6.4,0.54,0
2,3.0,-0.010997,3.050239,120.0,156.0,191.0,229.0,283.6,12.908488,0.066222,...,White,Some college but no degree,NaN,NaN,62500.0,0.023,97,8.9,0.58,1
3,4.0,0.007940,0.426055,68.3,86.0,100.0,120.0,149.0,8.876556,0.083210,...,White,Bachelor's Degree,NaN,NaN,62500.0,0.735,119,5.6,0.89,0
4,5.0,-0.001418,0.223682,119.0,151.0,203.0,264.0,313.0,12.079348,0.057533,...,White,Master's Degree,NaN,NaN,87500.0,0.626,264,9.1,0.75,1


In [13]:
pd.set_option('display.max_columns', None)
print(df_metrics.describe(include='all'))

              PtID  slope_TIR_per_day  slope_mean_glucose_per_day  \
count   200.000000         200.000000                  200.000000   
unique         NaN                NaN                         NaN   
top            NaN                NaN                         NaN   
freq           NaN                NaN                         NaN   
mean    101.015000           0.000168                    0.466607   
std      58.511991           0.019340                    3.578325   
min       1.000000          -0.043990                  -11.777586   
25%      50.750000          -0.010962                   -1.402099   
50%     100.500000          -0.000152                    0.272642   
75%     151.250000           0.007587                    2.092105   
max     203.000000           0.141364                   14.536088   

        glucose_p10  glucose_p25  glucose_p50  glucose_p75  glucose_p90  \
count    200.000000   200.000000   200.000000   200.000000   200.000000   
unique          NaN  

In [14]:
df_metrics.to_csv('df_metrics_CGM_biochem_demographics_extval.csv', index=False)